# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
# Define CSV
weather_data_file = "../output_data/weather_data.csv"

# Load into 
weather_data_df = pd.read_csv(weather_data_file)
weather_data_df.head()


,Unnamed: 0,City,Cloudiness,Date,Temp,Wind Speed,Humidity,Country,Lat,Long
0,0,port lincoln,100,1970-01-01 00:00:01.596660445,49.86,19.30,68,AU,-34.73,135.87
1,1,hermanus,42,1970-01-01 00:00:01.596659919,43.00,1.01,94,ZA,-34.42,19.23
2,2,verkhniy avzyan,48,1970-01-01 00:00:01.596660446,62.78,5.37,83,RU,53.53,57.54
3,3,chokurdakh,13,1970-01-01 00:00:01.596659637,59.43,6.80,48,RU,70.63,147.92
4,4,olinda,40,1970-01-01 00:00:01.596659972,78.80,18.34,65,BR,-8.01,-34.86


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [35]:

gmaps.configure(api_key = g_key)
locations=weather_data_df[["Lat","Long"]]
humidity = weather_data_df.Humidity.astype(float)

fig=gmaps.figure()

heatmap_layer=gmaps.heatmap_layer(locations, weights=humidity, dissipating = False, max_intensity=50, point_radius=1)
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
# Max temperature of 80 degrees and no lower than 65 degrees, windspeed less than 10mph, and no cloudiness

conditions_df = pd.DataFrame(weather_data_df, columns = ["City", "Temp", "Wind Speed", "Cloudiness"])

temp = (conditions_df["Temp"] <= 80) & (new_df["Temp"] > 65)
wind_speed = conditions_df["Wind Speed"] < 10
cloudiness = conditions_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
conditions_df[temp & wind_speed & cloudiness]

,City,Temp,Wind Speed,Cloudiness
59,tiznit,67.28,3.36,0
63,arcachon,77.00,8.05,0
85,lata,78.01,5.01,0
158,saint-francois,71.60,3.36,0
172,tripoli,77.52,9.01,0
177,rio grande,78.80,8.05,0
178,shimoda,73.00,1.01,0
206,chuy,66.02,7.99,0
243,paray-le-monial,71.60,3.36,0
246,goya,79.59,6.42,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
# Create hotel dataframe 
hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country", "Lat", "Long"])
hotel_df["Hotel Name"] = ""
hotel_df.dropna()

,City,Country,Lat,Long,Hotel Name
0,port lincoln,AU,-34.73,135.87,
1,hermanus,ZA,-34.42,19.23,
2,verkhniy avzyan,RU,53.53,57.54,
3,chokurdakh,RU,70.63,147.92,
4,olinda,BR,-8.01,-34.86,
...,...,...,...,...,...
542,beloomut,RU,54.94,39.34,
543,gamba,GA,-2.65,10.00,
544,beaverlodge,CA,55.22,-119.44,
545,balabac,PH,7.99,117.06,


In [31]:
#Create empty list for Hotel Name column and parameters to search
hotel_name=[]
target_type = "hotel"
radius = 5000

#Define parameters
params = {
    "types": target_type,
    "radius": radius,
    "key": g_key
}

#For loop to search for hotels in locations meeting weather criteria
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Long"]
    
    params["location"] = f"{lat},{lng}"
      
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response_json=response.json()

    try: 
        hotel_name.append(response_json['results'][0]['name'])
    
        hotel_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
    
        print(f"The nearest hotel to {row[0]} is {response_json['results'][0]['name']}")  
    
    except (KeyError, IndexError):
        print("No Hotel Found. Next!")


No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel Found. Next!
No Hotel F

In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [37]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))